In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model
from keras.layers import merge, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping

Test pobranego modelu w całości 

In [ ]:
from pathlib import Path
train_dir = "kaggle_bee_vs_wasp/"
bs = 32 # Batch size
resize_size = 224 # for training, resize all the images to a square of this size
training_subsample = 1 # for development, use a small fraction of the entire dataset rater than full dataset

bees_vs_wasps_dataset_path=Path(train_dir) # this is relative to the "example_notebook" folder. Modify this to reflect your setup
df_labels = pd.read_csv(bees_vs_wasps_dataset_path/'labels.csv')
df_labels=df_labels.set_index('id')
# perform dataset subsampling
df_labels = df_labels.sample(frac=training_subsample, axis=0)
insect_class = {'bee': 0,'wasp': 1, 'insect': 2,'other': 3} 
#df_labels.label = [insect_class[item] for item in df_labels.label] 
df_labels = df_labels[['path','label']]

In [ ]:
train_df, test_val_df = train_test_split(df_labels, test_size=0.2)
test_df, validation_df = train_test_split(test_val_df, test_size=0.5)
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)

In [ ]:
#Uncomment to read data from csv
#df_labels = pd.read_csv("Dataset_full.csv") #full dataset
#df_labels = pd.read_csv("Dataset_10precent.csv") #0.1 fraction for development

In [ ]:
#Create custom vgg model
image_input = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

model.summary()

last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
out = Dense(4, activation='softmax', name='output')(x)
custom_vgg_model = Model(image_input, out)

#train only 3 last layers
for layer in custom_vgg_model.layers[:-3]:
	layer.trainable = False

custom_vgg_model.compile(loss='sparse_categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
custom_vgg_model.summary()


In [ ]:
#Create generators from dataframes
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    train_dir,  
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=bs
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validation_df, 
    train_dir, 
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=bs
)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    train_dir, 
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=1
)

In [ ]:
#Early stopping
checkpoint = ModelCheckpoint(
    "classification_model_checkpoint.hdf5", 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=False,
    save_weights_only=False, 
    mode='auto', 
    save_freq='epoch',
    options=None
)
early = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=40,
    verbose=1,
    mode="auto",
)

In [ ]:
n_training_samples = len(train_df)
n_validation_samples = len(validation_df)

history = custom_vgg_model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=n_validation_samples//bs,
    steps_per_epoch=n_training_samples//bs,
    callbacks=[checkpoint, early])

In [ ]:
filenames = test_generator.filenames
n_test_samples = len(filenames)
predict = custom_vgg_model.predict(test_generator, steps=n_test_samples)
scores = custom_vgg_model.evaluate_generator(test_generator,steps=n_test_samples)

In [ ]:
custom_vgg_model.save('classifiaction_layer_trained_model')
np.savetxt("model1_class/val_loss.csv",  
           history.history["val_loss"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model1_class/val_accuracy.csv",  
           history.history["val_accuracy"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model1_class/loss.csv",  
           history.history["loss"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model1_class/accuracy.csv",  
           history.history["accuracy"], 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model1_class/accuracy_on_test.csv",  
           scores, 
           delimiter =", ",  
           fmt ='% s') 

In [ ]:
print ("Model accuracy on test set:" + str(scores[1]) )